In [2]:
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt 
import glob
import os
import xarray as xr 
import pandas as pd
import numpy as np


from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=12,                     # Number of cores per task (adjust if needed)
    processes=6,                 # One process per task
    memory="100GB",               # --mem
    walltime="01:15:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)


from dask.distributed import Client


cluster.scale(jobs=10)
client = Client(cluster)


Exception in callback multi_future.<locals>.callback() at /home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/tornado/gen.py:523
handle: <Handle multi_future.<locals>.callback() at /home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/tornado/gen.py:523>
Traceback (most recent call last):
  File "/home1/nalex2023/.conda/envs/nma/lib/python3.13/asyncio/events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/tornado/gen.py", line 529, in callback
    result_list.append(f.result())
                       ~~~~~~~~^^
  File "/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/bokeh/server/session.py", line 88, in _needs_document_lock_wrapper
    with await self._lock.acquire():
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
asyncio.exceptions.CancelledError
Exception in callback multi_future.<locals>.callback() at /home1/nalex2023/

In [73]:
client.close()
cluster.close()

In [ ]:
met_arm = '/home1/nalex2023/Datasets/met_manus/'


def create_dft(in_dir,arm_data=True):
    files = glob.glob(in_dir+os.sep+'*.nc')
    if len(files) == 0:
        files = glob.glob(in_dir+os.sep+'*.cdf')
        if len(files) == 0:
            raise Exception('No files Found')
        else:
            pass
    else:
        pass
    
    dft = pd.DataFrame(files,columns=['filepath'])
    dft['filename'] = dft['filepath'].str.split(os.sep).str[-1]
    
    if arm_data:
        dft['datetime'] = pd.to_datetime(dft['filename'].str.split(
            '.',
            expand=True).iloc[:, 2] + dft['filename'].str.split(
                '.',  expand=True).iloc[:, 3],format='%Y%m%d%H%M%S')


    return dft


dft_met = create_dft(met_arm).sort_values('datetime')


years = np.arange(2000,2013)

dft_sub = dft_met[dft_met['datetime'].dt.year.isin(years)]


dft_sub

,filepath,filename,datetime
2516,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000101.000000.cdf,2000-01-01 00:00:00
3481,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000102.000001.cdf,2000-01-02 00:00:01
867,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000103.000000.cdf,2000-01-03 00:00:00
1904,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000104.000001.cdf,2000-01-04 00:00:01
6043,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20000105.000000.cdf,2000-01-05 00:00:00
...,...,...,...
182,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20121227.000000.cdf,2012-12-27 00:00:00
3352,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20121228.000000.cdf,2012-12-28 00:00:00
2332,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20121229.000000.cdf,2012-12-29 00:00:00
5538,/home1/nalex2023/Datasets/met_manus/twpmetC1.b...,twpmetC1.b1.20121230.000000.cdf,2012-12-30 00:00:00


In [66]:
def prepro_met(ds):
    ds = ds[['temp_mean','qc_temp_mean','rh_mean','qc_rh_mean','wspd_arith_mean','qc_wspd_arith_mean','wdir_vec_mean','qc_wdir_vec_mean']]    
    
    return ds



In [67]:



met_dset = xr.open_mfdataset(dft_sub['filepath'].values,chunks={},
                             preprocess=prepro_met,parallel=True,combine='by_coords').compute()


/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 28.21 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [68]:
qc_vars = [i for i in met_dset.data_vars if 'qc' in i]



In [69]:
qc_vars = [i for i in met_dset.data_vars if 'qc' in i]

# if qc is not 0 then set to nan

for i in qc_vars:
    met_dset[i.split('qc_')[1]] = met_dset[i.split('qc_')[1]].where(met_dset[i] == 0)


met_dset_hourly = met_dset.resample(time='1h').mean().compute()

In [72]:
out_dir = '/home1/nalex2023/Datasets/LSB_OUTS/'

met_dset_hourly.to_netcdf(out_dir+'met_arm_hourly_2000_2013.nc')

In [74]:
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt 
import glob
import os
import xarray as xr 
import pandas as pd
import numpy as np


from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=12,                     # Number of cores per task (adjust if needed)
    processes=6,                 # One process per task
    memory="100GB",               # --mem
    walltime="01:15:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)


from dask.distributed import Client


cluster.scale(jobs=10)
client = Client(cluster)

met_data_file = '/home1/nalex2023/Datasets/LSB_OUTS/met_arm_hourly_2000_2013.nc'

out_dir = '/home1/nalex2023/Datasets/LSB_OUTS/'

dset_met = xr.open_dataset(met_data_file).compute()

#dset_met_diurnal_complete = dset_met.groupby('time.hour').mean()

#dset_met_diurnal_complete.to_netcdf(out_dir+'met_arm_diurnal_2000_2013_complete.nc')

#dset_met_diurnal_seasonal = dset_met.groupby('time.season').apply(lambda x: x.groupby('time.hour').mean())

In [75]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.42.239.61:8787/status,
Dashboard: http://10.42.239.61:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.42.239.61:38083,Workers: 0
Dashboard: http://10.42.239.61:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [76]:
dset_met_diurnal_complete = dset_met.groupby('time.hour').mean()



In [77]:
dset_met_diurnal_complete.to_netcdf(out_dir+'met_arm_diurnal_2000_2013_complete.nc')

In [78]:
dset_met_diurnal_seasonal = dset_met.groupby('time.season').apply(lambda x: x.groupby('time.hour').mean())

In [79]:
dset_met_diurnal_seasonal.to_netcdf(out_dir+'met_arm_diurnal_2000_2013_seasonal.nc')